In [4]:
os.getenv("JAVA_HOME")

'C:\\Program Files\\Java\\jdk1.8.0_131'

In [5]:
# Set the environment to use standalone spark
import os
import sys

def set_home():
    os.environ['SPARK_HOME']="C:\spark\spark"
    os.environ['JAVA_HOME']="C:/Program Files/Java/jre1.8.0_152"
    sys.path.append("c:\spark\spark\python")
    os.environ['HADOOP_HOME']="c:\spark\hadoop"
    sys.path.append("C:\spark\mongo-hadoop\spark\src\main\python")
def set_work():
    os.environ['SPARK_HOME']="D:/spark/spark"
    #os.environ['JAVA_HOME']="C:/Program Files/Java/jre1.8.0_151"
    sys.path.append("D:\spark\spark\python")
    os.environ['HADOOP_HOME']="D:\spark\hadoop"
    sys.path.append("D:\mongo-hadoop\spark\src\main\python")
    
set_work()
#set_home()

In [6]:
# This will help to print all the commands output of a single cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id
#import pyspark_cassandra
#from pyspark_cassandra import CassandraSparkContext
import pymongo_spark
pymongo_spark.activate()

from pyspark.sql.functions import rand
from pyspark.sql.functions import monotonically_increasing_id
from skimage.data import imread
import io
import numpy as np
from pyspark.sql.types import *
import pandas as pd

In [7]:
spark

In [8]:
try:
    conf = SparkConf(True).set("spark.cassandra.connection.host", "127.0.0.1")
    sc =  CassandraSparkContext(conf=conf)
except:
    pass
sqlContext = SQLContext(sc)

file = "C:/Users/kokumars/OneDrive - Microsoft/Kaggle/France-ECommerce/category_names.csv"

customSchema = StructType([ \
    StructField("category_id", IntegerType(), True), \
    StructField("category_level1", StringType(), True), \
    StructField("category_level2", StringType(), True), \
    StructField("category_level3", StringType(), True)])

df = sqlContext.read \
    .format('com.databricks.spark.csv') \
    .options(header='true') \
    .load(file, schema = customSchema)
# map the categories to numeric categories, so we can do one hot encoding.
class_df = df.select(['category_id']).sort('category_id').withColumn("id", monotonically_increasing_id())
num_classes = class_df.count()

In [9]:
file = "C:/Users/kokumars/OneDrive - Microsoft/Kaggle/France-ECommerce/category_names.csv"
df = pd.read_csv(file, header='infer')
# map the categories to numeric categories, so we can do one hot encoding.
df = df.sort_values('category_id')
df.insert(0, 'id', range(len(df)))
df = df[['id', 'category_id']]
num_classes = len(df)
num_classes

5270

In [10]:
cat2idx = {}
idx2cat = {}

for ir in df.itertuples():
    category_id = ir[2]
    category_idx = ir[1]
    cat2idx[category_id] = category_idx
    idx2cat[category_idx] = category_id

In [11]:
# BSONFileRDD is coming from the pymongo_spark. It provides API to parse bson files to RDD
#file = 'C:/Users/kokumars/OneDrive - Microsoft/Kaggle/France-ECommerce/train_example.bson'
#file = 'C:/Users/kokumars/Desktop/Kaggle/train.bson'
file = "D:/Kaggle/France-ECommerce/train.bson"
config = None
%time rdd = sc.BSONFileRDD(file)
# Lets count how many products are present
# bsonFileRDD.count()
p = rdd.getNumPartitions()
#bsonFileRDD = bsonFileRDD.repartition(p)
rdd.getNumPartitions()
bsonFileRDD = rdd.repartition(p * 15)
bsonFileRDD.getNumPartitions()

Wall time: 1.3 s


1863

27945

In [12]:
# flatten the product list over the images of a product.
flatMapRDD = bsonFileRDD.map(lambda x: ((x['_id'], x['category_id']), x['imgs'])).flatMapValues(lambda x : x)
#fd = flatMapRDD.map(lambda x: Row(id=x[0][0], cid=x[0][1], \
#                                  img = (img_to_array(load_img(io.BytesIO(x[1]['picture'])))).astype('float32')/255.0))

fd = flatMapRDD.map(lambda x: Row(idx = x[0][0], cid =int(cat2idx[x[0][1]]), img = x[1]['picture']))
fd = fd.persist()

df = fd.toDF()
#_max = df.select('idx').rdd.max()
#_min = df.select('idx').rdd.min()
#_count = df.count()

df.registerTempTable("df_table")
t = sqlContext.sql("SELECT MAX('idx') as maxval FROM df_table").collect() #[0].asDict()['maxval']

In [13]:
train, val = fd.randomSplit([0.7, 0.3], seed=99)

In [14]:
sc

<SparkContext master=spark://10.171.23.123:7077 appName=PySparkShell>

In [ ]:
#train_df = train_df.withColumn("idx", monotonically_increasing_id())

In [ ]:
#val_df = val_df.withColumn("idx", monotonically_increasing_id())

In [ ]:
# Bad while running to put data to cassandra. Using different Data Model.
#train_df = train_df.sort('cid').withColumn("id", monotonically_increasing_id())
#val_df = val_df.sort('cid').withColumn("id", monotonically_increasing_id())

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

In [ ]:
import pandas as pd
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [ ]:
keyspace = "komal"
try:
    query = f"DROP KEYSPACE IF EXISTS {keyspace}"
    session.execute(query)
except:
    print("Drop keyspace {keyspace} failed")

In [ ]:
try:
    #query = "DROP KEYSPACE komal1;"
    #session.execute(query);
    query = "CREATE KEYSPACE IF NOT EXISTS komal WITH replication = {'class':'SimpleStrategy', 'replication_factor':1};"    
    session.execute(query);
except:
    print("Create keyspace already exists")
session.execute(f"USE {keyspace};")

In [ ]:
try:
    drop_table = "DROP TABLE IF EXISTS train"    
    session.execute(drop_table)
except:
    print("Drop Table train failed")

In [ ]:
try:
    drop_table = "DROP TABLE IF EXISTS val"
    session.execute(drop_table)
except:
    print("Drop table val failed")

In [ ]:
try:
    drop_table = "DROP TABLE IF EXISTS test_val"
    session.execute(drop_table)
except:
    print("Drop table test_val failed")

In [ ]:
try:
    drop_table = "DROP TABLE IF EXISTS test_train"
    session.execute(drop_table)
except:
    print("Drop table test_train failed")

In [ ]:
try:
    #"CREATE TABLE t (pk int, t int, v text, s text, PRIMARY KEY (pk, t));
    create_table = "CREATE TABLE train (cid BIGINT, img BLOB, PRIMARY KEY (cid, img));"                                        
    session.execute(create_table)
except:
    print("create table train failed")

In [ ]:
try:
    create_table = "CREATE TABLE val (cid BIGINT, img BLOB, PRIMARY KEY (cid, img));"
    session.execute(create_table)
except:
    print("Create Table Val failed")

In [ ]:
try:
    create_table = "CREATE TABLE test_val (cid BIGINT, img BLOB, PRIMARY KEY (cid));"
    session.execute(create_table)
except:
    print("Create Table Val failed")

In [ ]:
try:
    create_table = "CREATE TABLE test_train (cid BIGINT, img BLOB, PRIMARY KEY (cid));"
    session.execute(create_table)
except:
    print("Create Table test_train failed")

In [ ]:
session.set_keyspace(keyspace)
#result = session.execute("SELECT * FROM train;")
sc

In [ ]:
#fd.saveToCassandra("")

rdd = sc.cassandraTable(keyspace, "train")
rdd.count()
#session.execute("SELECT * FROM train")
#df.rdd.saveToCassandra("kaggle", "train")

In [ ]:
keyspace

In [ ]:
# Use it when u want to overwrite the entries .option("confirm.truncate", True) \
# Inputsize/ Records 58.2 GB / 7069896 *0.7
#train records: 8660954 
train_df.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append') \
    .option("table", "train") \
    .option("keyspace", "komal")\
    .save()

In [ ]:
# Input Size/Records 53.2 GB / 7069896 *0.3
#val_df records: 3710339
val_df.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append') \
    .option("table", "val") \
    .option("keyspace", "komal")\
    .save()

In [ ]:
train_df.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('overwrite') \
    .option("table", "test_train") \
    .option("keyspace", "kaggle")\
    .option("confirm.truncate", True) \
    .save()

In [ ]:
val_df.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('overwrite') \
    .option("table", "test_val") \
    .option("keyspace", keyspace)\
    .option("confirm.truncate", True) \
    .save()

create_table = """CREATE TABLE train (
    id BIGINT,
    cid BIGINT,
    img BLOB,
    PRIMARY KEY (id, cid));"""
session.execute(create_table)

In [ ]:
fd.count()

In [ ]:
8660954 + 3710339

In [ ]:
import pandas as pd
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [ ]:
_min = 20
_max = 30
query = f"SELECT * FROM val WHERE id < {_max} and id > {_min} LIMIT 100 ALLOW FILTERING;"
query
%time result = session.execute(query=query, timeout=50000000000)

In [ ]:
pdf = result._current_rows

In [ ]:
import pyspark_cassandra
rdd = sc.cassandraTable("kaggle", "train", row_format= pyspark_cassandra.RowFormat.ROW)

In [ ]:
# This is not effecient 
#rdd.filter("id < 100").collect()

In [ ]:
query = "CREATE TABLE IF NOT EXISTS CATEGORY( \
                category_id BIGINT, \
                category_level1 BIGINT, \
                category_level2 BIGINT, \
                category_level3 BIGINT, \
                PRIMARY KEY (category_id));"
session.execute(query)

In [ ]:
train_df.count()
#val_df.count()

In [ ]:
#s = pdf['cid']

In [ ]:
j = s.map(lambda x: 2)

In [ ]:
result = session.execute("SELECT * FROM TEST_TRAIN;", timeout=50000000000)

In [ ]:
#result.current_rows

In [ ]:
#result._current_rows

In [ ]:
rdd = sc.cassandraTable("kaggle", "train")

In [ ]:
df = rdd.toDF()

In [ ]:
tf = df.select('id')

In [ ]:
c= df['id']

In [ ]:
t = rdd.map(lambda x: (x['id'], 1))
#.groupbykey(lambda a,b : a+b).collect()

In [ ]:
t.reduceByKey(lambda a,b : a+b).count()

In [ ]:
df.show()

In [ ]:
train_df.show()

In [ ]:
rdd = sc.cassandraTable("kaggle", "test_train")

In [ ]:
rdd.toDF().sort('idx').show(100)

In [ ]:
rdd = sc.cassandraTable("kaggle", "train")

In [ ]:
rdd.cassandraCount()

In [ ]:
query = "select count(*) from train;"
result = session.execute(query, timeout=5000000000000)

In [ ]:
result.current_rows

In [ ]:
query = "select MAX(cid) from train;"
result = session.execute(query, timeout=50000000000000)

In [ ]:
result.current_rows

In [ ]:
query = "select MIN(cid) from train;"
result = session.execute(query, timeout=50000000000000)

In [ ]:
result.current_rows

In [ ]:
query = "select count(*) from train where cid > 4000 and cid < 5269 ALLOW FILTERING;"
result = session.execute(query, timeout=50000000)

In [ ]:
import pandas as pd
pd.DataFrame(result.current_rows)

In [ ]:
query = "select * from train where idx > 5000 and idx <= 6000 ALLOW FILTERING;"
result = session.execute(query, timeout=50000000)

In [ ]:
result.current_rows

In [ ]:
rd = sc.parallelize(fd.take(100000))

In [ ]:
rd.toDF().withColumn("idx", monotonically_increasing_id()).show()

In [ ]:
pt = train.getNumPartitions()
pv = val.getNumPartitions()
train = train.repartition(pt * 9)
val = val.repartition(pv * 9)
train.getNumPartitions()
val.getNumPartitions()

In [41]:
def set_work():
    return "file:///komal1234/Kaggle/test2/"
def set_home():
    return "C:/Users/kokumars/Desktop/Kaggle/train"

In [42]:
import getpass
getpass.getuser()

'kokumars'

In [ ]:
train.saveAsTextFile(set_work())

In [ ]:
val.saveAsTextFile("C:/Users/kokumars/Desktop/Kaggle/val", "org.apache.hadoop.io.compress.GzipCodec")

In [ ]:
train.getNumPartitions()

In [ ]:
val.getNumPartitions()

In [ ]:
file = "C:/Users/kokumars/Desktop/Kaggle/train/part-00000*"
rdd = sc.textFile(file)

In [ ]:
file = "C:/Users/kokumars/Desktop/Kaggle/train_test"
train.saveAsTextFile(file, "org.apache.hadoop.io.compress.GzipCodec")

In [ ]:
file = "C:/Users/kokumars/Desktop/Kaggle/train_test/part-00000.*"
rdd = sc.textFile(file)

In [ ]:
from pyspark.sql import Row
def ev(x):
    val = eval(x).asDict()
    return Row(idx = val['idx'], cid=val['cid'], img=val['img'])
    
r = rdd.map(ev)

r.toDF().count()

In [ ]:
x = val.asDict()
x['idx']

In [ ]:

"komal{0:05d}".format(idx)

In [ ]:
x.collect()